In [1]:
import os
import collections
import csv
import re #Regular Expressions
import operator # Majorly for sorting tf-idf dictionary on basis of values
import nltk
from nltk.corpus import stopwords # For removing stop words, use ntlk.download() to completely install this package
import pandas as pd
import numpy as np
from sklearn import model_selection as cv
from sklearn import naive_bayes
from sklearn.metrics import classification_report, confusion_matrix

# Parameters to customize according to use case

In [2]:
base_dir = "/home/aakash/Drive/Dropbox/ML/data/20_newsgroups/" #Set  this according to your machine
class_names = os.listdir(base_dir)

In [3]:
#Use small numbers for first time, articles =  100, classes = 2 , features = 100 
max_articles_of_each_class = 1000
no_of_classes = 10
no_of_features = 5000

# Caclulating tf-idf metric for articles of selected classes

In [4]:
def preprocess_file_text(file):
    text = file.read().lower() # Converting all to lowercase as lowercase and uppercase words should be considered same word
    text = re.sub('[^A-Za-z ]+', '', text) # Removing non-aplha characters
    text = re.sub('\s+', ' ', text)  # Condense all whitespace
    return text

In [5]:
words = set()
tf = {}
idf = {}
selected_classes = []
articles_read = {} # To keep track of which all articles were read while extracting features
for i in range(0,len(class_names)):
    if(i >= no_of_classes):
        break
    current_class = class_names[i]
    
    # 2 updates for tracking
    selected_classes.append(current_class)
    articles_read[current_class] = []
    
    class_dir = base_dir + current_class
    all_articles = os.listdir(class_dir)
    for j in range(0,len(all_articles)):
        if(j >= max_articles_of_each_class):
            break
        current_file = class_dir + "/" + all_articles[j]
        articles_read[current_class].append(all_articles[j]) 
        file = open(current_file, encoding = "ISO-8859-1")
        text = preprocess_file_text(file)
        file.close() # Always close a file after using it to free up system resources
        file_words = text.split()
        
        # Updating term-frequency dictionary
        word_count = collections.Counter(file_words)
        for word,freq in word_count.items():
            if(word in tf):
                tf[word] = tf[word] + freq
            else:
                tf[word] = freq
        
        #Updating (inverse document frequency) dictionary
        word_set = set(file_words)
        for word in word_set:
            if(word in idf):
                idf[word] = idf[word] + 1
            else:
                idf[word] = 1

In [6]:
tf_by_idf = {}
for key in tf.keys():
    tf_by_idf[key] = tf[key]/idf[key]

# Removing Stop Words from data dictionary

In [7]:
for stop_word in stopwords.words("english"):
    if(stop_word in tf_by_idf.keys()):
        tf_by_idf.pop(stop_word)

# Selecting top x words with max tf-idf value as Features

In [8]:
tf_by_idf = sorted(tf_by_idf.items(), key=operator.itemgetter(1))
tf_by_idf.reverse()
#tf_by_idf

In [9]:
features = set()
for i in range(0,no_of_features):
    features.add(tf_by_idf[i][0])

# Creating Dataframe by reading the articles again on the basis of selected features

In [10]:
# It takes time,so be patient (Reduce feature count to reduce this time but score will be affected accordingly)

columns = list(features)
total_articles_to_process = 0
for class_name in articles_read.keys():
    total_articles_to_process += len(articles_read[class_name])
data = []

articles_processed = 0
for current_class in selected_classes:
    class_dir = base_dir + current_class
    for article in articles_read[current_class]:
        articles_processed += 1
        if(articles_processed%500 == 0):
            print(articles_processed,"articles are processed out of",total_articles_to_process,"articles")
        current_file = class_dir + "/" + article
        file = open(current_file, encoding = "ISO-8859-1")
        text = preprocess_file_text(file)
        file.close() # Always close a file after using it to free up system resources
        file_words = text.split()
        
        word_count = collections.Counter(file_words)
        training_data = [0]*(len(columns) + 1) # +1 because last column is of output class
        for i in range(0,len(columns)): 
            feature = columns[i]
            if(feature in word_count.keys()):
                training_data[i] = word_count[feature]
        training_data[-1] = current_class
        data.append(training_data)

columns.append("class")
df = pd.DataFrame(data,columns=columns)

500 articles are processed out of 10000 articles
1000 articles are processed out of 10000 articles
1500 articles are processed out of 10000 articles
2000 articles are processed out of 10000 articles
2500 articles are processed out of 10000 articles
3000 articles are processed out of 10000 articles
3500 articles are processed out of 10000 articles
4000 articles are processed out of 10000 articles
4500 articles are processed out of 10000 articles
5000 articles are processed out of 10000 articles
5500 articles are processed out of 10000 articles
6000 articles are processed out of 10000 articles
6500 articles are processed out of 10000 articles
7000 articles are processed out of 10000 articles
7500 articles are processed out of 10000 articles
8000 articles are processed out of 10000 articles
8500 articles are processed out of 10000 articles
9000 articles are processed out of 10000 articles
9500 articles are processed out of 10000 articles
10000 articles are processed out of 10000 articles


In [11]:
# To save created df to csv file
#df.to_csv("data_" + str(no_of_classes) + "classes.csv",encoding="UTF-8", index=False)

In [12]:
# To load df from saved file
#df = pd.read_csv("data_" + str(no_of_classes) + "classes.csv",index_col=False)

In [13]:
X = df.values[:,:-1]
Y = df.values[:,-1]

In [ ]:
X_train,X_test,Y_train,Y_test = cv.train_test_split(X,Y,test_size=0.05,random_state=0)

# Using sklearn

### Multinomial Naive Bayes

In [ ]:
mnb = naive_bayes.MultinomialNB(alpha=0.1)
mnb.fit(X_train,Y_train)
Y_pred = mnb.predict(X_test)
print("Size of Y_test = ",len(Y_test)," and wrong results = ",(Y_pred != Y_test).sum())
print("Score is ",mnb.score(X_test,Y_test))

### Gaussian Naive Bayes

In [ ]:
gnb = naive_bayes.GaussianNB()
gnb.fit(X_train,Y_train)
Y_pred = gnb.predict(X_test)
print("Size of Y_test = ",len(Y_test)," and wrong results = ",(Y_pred != Y_test).sum())
print("Score is ",gnb.score(X_test,Y_test))

# Using My Implementation

### Gaussian Model

In [ ]:
def calculatePriorProbabilities(Y):
    classes = set(Y)
    result = {}
    for i in classes:
        result[i] = (len(Y[Y==i])/len(Y))
    return result

In [ ]:
#Returns a dictionary
# Structure of dictionary {class_names : { keys are features + priorProbaility : Value is hash containing mean and variance of feature of those class samples}}
def fitGaussianBayes(X_train,Y_train,priorProbabilities={}):
    result = {}
    output_classes = set(Y_train)
    if (len(priorProbabilities) != len(output_classes)) :
        priorProbabilities = calculatePriorProbabilities(Y_train)
    
    #This epsilon is added to variance of each feature as zero variance will cause numeric errors
    epsilon = 1e-9 * np.var(X_train,axis=0).max()
    
    for current_class in output_classes:
        value = {}
        result[current_class] = value
        class_samples = (Y_train == current_class)
        Y_train_current = Y_train[class_samples]
        X_train_current = X_train[class_samples]
        for feature in range(0,X_train.shape[-1]):
            #Since each feature is a Gaussian Distribution, we need to store mean and variance of those samples only
            value[feature] = {}
            feature_hash = value[feature]
            feature_hash["mean"] = X_train_current[:,feature].mean()
            feature_hash["var"] = X_train_current[:,feature].var() + epsilon
        value["priorProbability"] = priorProbabilities[current_class]
    return result

In [ ]:
#Returns log of gaussian probability
def logGPUsingMean(mean,variance,x):
# Using commented approach can have cases with (x-mean)**2 having greater value than 700 and thus nr in prob variable will become nearly zero and will take log to inf
#     power = -((x - mean)**2)/(2*variance)
#     prob = np.exp(power)/np.sqrt(2*np.pi*variance)
#     return np.log(prob)
    logProb = - 0.5 * ((x - mean)**2)/variance # -0.5 because of 2 in denominator of power and - in numerator which were omitted in expression
    logProb = logProb - 0.5 * np.log(2*np.pi*variance)
    return logProb

In [ ]:
def predictGaussianClassProbabiltyUsingDictionary(dictionary,X_test_sample):
    result = np.log(dictionary["priorProbability"])
    for i in range(0,len(dictionary.keys())-1): #-1 because last key is for probabilty, and each key is a feature
        logGP = logGPUsingMean(dictionary[i]["mean"],dictionary[i]["var"],X_test_sample[i])
        result = result + (logGP)
    return result

In [ ]:
def predictGaussianBayes(dictionary,X_test):
    classes = set(dictionary.keys())
    test_samples = X_test.shape[0]
    y_pred = [0] * test_samples
    
    for i in range(0,test_samples):
        probabilities = {}
        for current_class in classes:
            probabilities[current_class] = predictGaussianClassProbabiltyUsingDictionary(dictionary[current_class],X_test[i,:])
        #print("For sample i = ",i," probabilities are = ",probabilities)
        y_pred[i] = max(probabilities,key=probabilities.get)
    return y_pred

In [ ]:
dictionary = fitGaussianBayes(X_train,Y_train)

In [ ]:
Y_pred = predictGaussianBayes(dictionary,X_test)
print("Size of Y_test = ",len(Y_test)," and wrong results = ",(Y_pred != Y_test).sum())
print("Score is ",(Y_pred == Y_test).sum()/(len(Y_test)))

### Using Multinomial Model

In [ ]:
#Returns a dictionary
# Structure of dictionary {class_names : { keys are index of features as per df, prior Probaility of class, correctedDenominator for calculating probability : Value for each feature is sum of all frequencies of that feature in training samples of that class.}}
def fitMultinomialBayes(X_train,Y_train,priorProbabilities={},correctionFactor=1):
    result = {}
    output_classes = set(Y_train)
    featureCount = X_train.shape[1]
    correctionFactor = max(1e-10,correctionFactor) # To prevent absolute zeros
    if (len(priorProbabilities) != len(output_classes)) :
        priorProbabilities = calculatePriorProbabilities(Y_train)
        
    for current_class in output_classes:
        value = {}
        result[current_class] = value
        class_samples = (Y_train == current_class)
        Y_train_current = Y_train[class_samples]
        X_train_current = X_train[class_samples]
        value["priorProbability"] = priorProbabilities[current_class]
        value["correctedDr"] = X_train_current.sum() + (correctionFactor*featureCount)# Sum of frequencies of all words that appear in the articles belonging to current class
        for feature in range(0,featureCount):
            value[feature] = X_train_current[:,feature].sum() + correctionFactor
    return result

In [ ]:
def logMultinomialProbability(nr,dr,X):
    return X*np.log((nr/dr))

In [ ]:
def predictMultinomialClassProbabiltyUsingDictionary(dictionary,X_test_sample):
    result = np.log(dictionary["priorProbability"])
    dr = dictionary["correctedDr"]
    for i in range(0,len(dictionary.keys())-2): #-2 because 2 keys are for priorProbabilty and correctedDenominator and rest of the keys are features
        logMP = logMultinomialProbability(dictionary[i],dr,X_test_sample[i])
        result = result + (logMP)
    return result

In [ ]:
def predictMultinomialBayes(dictionary,X_test):
    classes = set(dictionary.keys())
    test_samples = X_test.shape[0]
    y_pred = [0] * test_samples
    
    for i in range(0,test_samples):
        probabilities = {}
        for current_class in classes:
            probabilities[current_class] = predictMultinomialClassProbabiltyUsingDictionary(dictionary[current_class],X_test[i,:])
        #print("For sample i = ",i," probabilities are = ",probabilities)
        y_pred[i] = max(probabilities,key=probabilities.get)
    return y_pred

In [ ]:
dictionary = fitMultinomialBayes(X_train,Y_train,correctionFactor=0.1)

In [ ]:
Y_pred = predictMultinomialBayes(dictionary,X_test)
print("Size of Y_test = ",len(Y_test)," and wrong results = ",(Y_pred != Y_test).sum())
print("Score is ",(Y_pred == Y_test).sum()/(len(Y_test)))

# Observations

### Use log likelihood of classes, otherwise most of probabilities will turn out to be so small that it would eventually become zero and can't be compared for different classes.
### Issue occurs if order of size of dataframe becomes more than 10^7. So, how to deal with 20 classes with 1000 articles each if we consider atleast 1000 featues, the size will be of order 2*10^7. Algo will perform much better with around 10^4 feature count. So, for inbuilt NB, use partial_fit method on chunks of data. 
### For correction in Gaussian Bayes Theorem, we can add epsilon to variance inorder to prevent divison by zero which causes numeric errors. 